In [ ]:
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
roi = Map.draw_last_feature.geometry()

In [ ]:
def clip(img):
    return img.clip(roi)

In [ ]:
images = ee.ImageCollection("NASA/GIMMS/3GV0").select('ndvi')\
        .filterDate('1982', '2013').map(clip)

In [ ]:
visParam = {
 'min': -1,
 'max': 1,
 'palette': ['000000', 'f5f5f5', '119701']
}
Map.addLayer(images, visParam, 'NDVI')

In [ ]:
def savitzky_golay(y, window_size, order):
    half_window = (window_size - 1) / 2
    deriv = 0
    order_range = ee.List.sequence(0, order)
    k_range = ee.List.sequence(-half_window, half_window)
    def fun1(k):
        return order_range.map(lambda o: ee.Number(k).pow(o))
    b = ee.Array(k_range.map(fun1))
    mPI = ee.Array(b.matrixPseudoInverse())
    impulse_response = (mPI.slice(**{ 'axis': 0, 'start': deriv, 'end': deriv + 1 })).project([1])
    y0 = y.get(0)
    firstvals = y.slice(1, half_window + 1).reverse().map(
        lambda e: ee.Number(e).subtract(y0).abs().multiply(-1).add(y0))
    yend = y.get(-1)
    lastvals = y.slice(-half_window - 1, -1).reverse().map(
        lambda e: ee.Number(e).subtract(yend).abs().add(yend))
    y_ext = firstvals.cat(y).cat(lastvals)
    runLength = ee.List.sequence(0, y_ext.length().subtract(window_size))
    smooth = runLength.map(
        lambda i:ee.Array(y_ext.slice(ee.Number(i), ee.Number(i).add(window_size))).multiply(impulse_response).reduce("sum", [0]).get([0])
    )
    return smooth

In [ ]:
def sg_images(images:ee.ImageCollection, order:int, window_size:int) -> list:
    half_window = (window_size - 1) / 2
    deriv = 0
    order_range = ee.List.sequence(0, order)
    k_range = ee.List.sequence(-half_window, half_window)
    def fun1(k):
        return order_range.map(lambda o: ee.Number(k).pow(o))
    b = ee.Array(k_range.map(fun1))
    mPI = ee.Array(b.matrixPseudoInverse())
    impulse_response = (mPI.slice(**{ 'axis': 0, 'start': deriv, 'end': deriv + 1 })).project([1])

    y = images.sort('system:time_start', False).toBands().toArray()
    times =images.aggregate_array('system:time_start')
    ids = images.aggregate_array('system:id')
    y1 = images.sort('system:time_start', True).toBands().toArray()
    y0 = y1.arrayGet(0)
    firstvals = y.arraySlice(0, -half_window - 1, -1).subtract(y0).abs().multiply(-1).add(y0)
    yend = y.arrayGet(0)
    lastvals = y.arraySlice(0, 1, half_window+1).subtract(yend).abs().add(yend)
    y_ext = firstvals.arrayCat(y1, 0).arrayCat(lastvals, 0)
    runLength = ee.List.sequence(0, images.size().subtract(1))

    smooth = []
    for i in runLength.getInfo():
        smooth.append(y_ext.arraySlice(0, ee.Number(i), ee.Number(i).add(window_size))
                      .multiply(impulse_response).arrayReduce("sum", [0]).arrayGet([0])
                     .set({'system:time_start':times.get(i), 'system:id':ids.get(i)}))
    return smooth

In [ ]:
sg = sg_images(images, 2, 9)
Map.addLayer(images.first(), {}, 'y1')
Map.addLayer(sg[0], {}, 'sg_')

In [ ]:
Map.addLayer(images.first(), visParam, 'y1')
Map.addLayer(sg[-1], visParam, 'sg_')